https://huggingface.co/docs/transformers/v4.44.2/en/llm_tutorial

<div class="markdown-google-sans">
  <h2>Generation with LLMs</h2>
</div>

LLMs, or Large Language Models, are the key component behind text generation. In a nutshell, they consist of large pretrained transformer models trained to predict the next word (or, more precisely, token) given some input text. Since they predict one token at a time, you need to do something more elaborate to generate new sentences other than just calling the model — you need to do autoregressive generation.

Autoregressive generation is the inference-time procedure of iteratively calling a model with its own generated outputs, given a few initial inputs. In 🤗 Transformers, this is handled by the generate() method, which is available to all models with generative capabilities.

This tutorial will show you how to:
- Generate text with an LLM
- Avoid common pitfalls
- Next steps to help you get the most out of your LLM

Before you begin, make sure you have all the necessary libraries installed:

<div class="markdown-google-sans">
  <h2>Generation with LLMs</h2>
</div>

LLM 또는 대규모 언어 모델은 텍스트 생성의 핵심 구성 요소입니다. 간단히 말해, 일부 입력 텍스트가 주어지면 다음 단어(또는 더 정확히는 토큰)를 예측하도록 훈련된 대규모 사전 훈련된 transformer models로 구성됩니다. 한 번에 하나의 토큰을 예측하므로 모델을 호출하는 것 외에도 새로운 문장을 생성하기 위해 더 정교한 작업을 수행해야 합니다. 즉, autoregressive generation을 수행해야 합니다.

autoregressive generation은 몇 가지 초기 입력이 주어지면 자체 생성된 출력이 있는 모델을 반복적으로 호출하는 inference-time 절차입니다. 🤗 Transformers에서 이는 생성 기능이 있는 모든 모델에서 사용할 수 있는 generate() 메서드로 처리됩니다.

이 튜토리얼에서는 다음 방법을 보여줍니다.
- LLM으로 텍스트 생성
- 일반적인 함정 피하기
- LLM을 최대한 활용하는 데 도움이 되는 다음 단계

시작하기 전에 필요한 모든 라이브러리가 설치되어 있는지 확인하세요.

In [ ]:
! pip install transformers bitsandbytes

<div class="markdown-google-sans">

## **Generate text**
</div>

A language model trained for causal language modeling takes a sequence of text tokens as input and returns the probability distribution for the next token.  

인과적 언어 모델링(causal language modeling)을 목적으로 학습된 언어 모델은 일련의 텍스트 토큰을 입력으로 사용하고, 그 결과로 다음 토큰이 나올 확률 분포를 제공합니다.

In [ ]:
from IPython.display import HTML

HTML("""
<video width=400 controls>
      <source src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/assisted-generation/gif_1_1080p.mov" type="video/mp4">
</video>
""")

A critical aspect of autoregressive generation with LLMs is how to select the next token from this probability distribution. Anything goes in this step as long as you end up with a token for the next iteration. This means it can be as simple as selecting the most likely token from the probability distribution or as complex as applying a dozen transformations before sampling from the resulting distribution.



LLM과 자기회귀 생성을 함께 사용할 때 핵심적인 부분은 이 확률 분포로부터 다음 토큰을 어떻게 고를 것인지입니다. 다음 반복 과정에 사용될 토큰을 결정하는 한, 어떠한 방법도 가능합니다. 확률 분포에서 가장 가능성이 높은 토큰을 선택하는 것처럼 간단할 수도 있고, 결과 분포에서 샘플링하기 전에 수십 가지 변환을 적용하는 것처럼 복잡할 수도 있습니다.

In [ ]:
from IPython.display import HTML

HTML("""
<video width=400 controls>
      <source src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/assisted-generation/gif_2_1080p.mov" type="video/mp4">
</video>
""")

The process depicted above is repeated iteratively until some stopping condition is reached. Ideally, the stopping condition is dictated by the model, which should learn when to output an end-of-sequence (EOS) token. If this is not the case, generation stops when some predefined maximum length is reached.

Properly setting up the token selection step and the stopping condition is essential to make your model behave as you’d expect on your task. That is why we have a GenerationConfig file associated with each model, which contains a good default generative parameterization and is loaded alongside your model.

위에서 설명한 과정은 어떤 종료 조건이 충족될 때까지 반복적으로 수행됩니다. 모델이 시퀀스의 끝(EOS 토큰)을 출력할 때까지를 종료 조건으로 하는 것이 이상적입니다. 그렇지 않은 경우에는 미리 정의된 최대 길이에 도달했을 때 생성이 중단됩니다.

모델이 예상대로 동작하기 위해선 토큰 선택 단계와 정지 조건을 올바르게 설정하는 것이 중요합니다. 이러한 이유로, 각 모델에는 기본 생성 설정이 잘 정의된 GenerationConfig 파일이 함께 제공됩니다.

##First, you need to load the model.

https://huggingface.co/collections/Qwen/qwen25-66e81a666513e518adb90d9e

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-1.5B", device_map="auto", load_in_4bit=True
)

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

You’ll notice two flags in the from_pretrained call:

- device_map ensures the model is moved to your GPU(s)
- load_in_4bit applies 4-bit dynamic quantization to massively reduce the resource requirements

There are other ways to initialize a model, but this is a good baseline to begin with an LLM.



## Next, you need to preprocess your text input with a tokenizer.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B", padding_side="left")
model_inputs = tokenizer(["A list of colors: red, blue"], return_tensors="pt").to("cuda")
# model_inputs = tokenizer(["색깔 리스트: 빨강, 블루"], return_tensors="pt").to("cuda")

In [ ]:
print(model_inputs)

In [ ]:
generated_ids = model.generate(**model_inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens=50)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

do_sample=True

In [ ]:
generated_ids = model.generate(**model_inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens=50, do_sample=True)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

# Instruct


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-1.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
prompt = "Give me a short introduction to large language model."

messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

In [ ]:
def make_llm_response(prompt):
    messages = [
        {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

while True:
    question = input("User: ")
    if question == "exit":
        break
    response = make_llm_response(question)
    print("\nAssistant: " + response + "\n")
